In [1]:
import random
import numpy as np
from collections import deque
import pandas as pd 

# Parámetros

In [2]:
#Horizonte de simulacion (Turnos)
Turnos = 1
Reloj = 0

In [3]:
#Se leen las caracteristicas de las palas
dfPalas = pd.read_excel('Informacion Palas.xlsx',  1)
InfoPalas = list(zip(dfPalas[dfPalas.columns[0]].values, dfPalas[dfPalas.columns[1]].values, dfPalas[dfPalas.columns[2]].values, \
                            dfPalas[dfPalas.columns[3]].values, dfPalas[dfPalas.columns[4]].values, \
                            dfPalas[dfPalas.columns[5]].values, dfPalas[dfPalas.columns[6]].values, \
                            dfPalas[dfPalas.columns[7]].values, dfPalas[dfPalas.columns[8]].values))

#Lista de nodos de mineral y esteril
Nodos_Mineral = ["P1", "P2", "P3", "P4", "P5"]
Nodos_Esteril = ["P6", "P7", "P8", "P9" ]

#Número de camiones disponibles
total_camiones = 81

#Lista de stocks/botaderos
#ID, espacios, ubicacion, tiempo_atencion ID=ubicacion?
SoB = ["S1", "S2", "B1", "B2"]
espacios = 5
tiempo_atencion_stock = 70 #segundos

#Se lee la informacion del chancador
dfChancador = pd.read_excel('Informacion Palas.xlsx',  0)
InfoChancador = list(zip(dfChancador[dfChancador.columns[0]].values, dfChancador[dfChancador.columns[1]].values, dfChancador[dfChancador.columns[2]].values, \
                            dfChancador[dfChancador.columns[3]].values, dfChancador[dfChancador.columns[4]].values, \
                            dfChancador[dfChancador.columns[5]].values, dfChancador[dfChancador.columns[6]].values, \
                            dfChancador[dfChancador.columns[7]].values, dfChancador[dfChancador.columns[8]].values))

# 1. Clases
## 1.1. Palas

In [4]:
class Pala:
    def __init__(self, ID, capacidad, ubicacion, media_recuperacion, media_falla, velocidad, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ubicacion = ubicacion #Nodo en que esta ubicada la pala
        self.capacidad = capacidad #Capacidad nominal de excavacion
        self.ID = ID #ID pala
        self.fila = deque() #camiones en fila para se atendidos
        self.aculatados = deque() #camiones aculatados 
        self.horas_eventos = {"Recuperacion":99999999, "Caida":None, "Inicio Movimiento":99999999, "Fin Movimiento":99999999} 
        self.proximo_evento = None #Nombre del evento mas proximo
        self.toneladas_sacadas = {} #Diccionario de tuplas con toneladas sacadas por posicion/nodo y ley
        self.puntaje = None #Puntaje para definir criticidad de la pala (Usado en Li por ej.)
        self.media_recuperacion = media_recuperacion
        self.media_falla = media_falla
        self.velocidad = velocidad
        self.TPH = {} #Diccionario con los resultados del PL
        self.tiempo_aculatado = tiempo_aculatado
        self.tiempo_atencion = None
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
               
            
    def caida(self, Reloj):#Completar comn PL y con Dinamico
        self.estado = 0
        #POR FUERA:
        #Se debe correr el PL y luego el dinámico para todos los camiones en la fila, y en camion hacia la pala
        #for cada camion if camion hacia pala.ID correr dinamico 
        self.horas_eventos["Caida"] = 99999999
        self.horas_eventos["Recuperacion"] = Reloj + self.tiempo_recuperacion()
        if self.horas_eventos["Recuperacion"] > self.horas_eventos["Inicio Movimiento"]:
            self.horas_eventos["Inicio Movimiento"] = self.horas_eventos["Recuperacion"] + 0.0001 #Postergo el movimiento hasta que se arregle la pala
        self.proximo_evento = "Recuperacion"
        return None
    
    def recuperacion(self, Reloj):
        self.estado = 1
        self.horas_eventos["Recuperacion"] = 99999999
        self.horas_eventos["Caida"] = Reloj + self.tiempo_falla()
        self.proximo_evento = min(horas_eventos, key=horas_eventos.get)
        return None
    
    def inicio_movimiento(self, Reloj, destino, MTV):
        pass
    
    def fin_movimiento(self, Reloj, Lista_movimientos_agendados):
        pass
       
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        return(min(horas_eventos.values()))
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        

## 1.2. Camiones

In [5]:
class Camion:
    def __init__(self, ID):
        self.ID = ID
        self.destino = None
        self.origen = None
        self.almuerzo = None #0: No almorzo, 1 primera colacion, 2 segunda colacion (solo aplica turno B)
        self.hora_siguiente = 0
        self.descargas = {} #Diccionario que almacena la historia de las descargas: key = (Origen, Destino)| value = toneladas
        self.carga = 0
        self.aculatandose = False
        self.estado = None #Viajando, Aculatandose, Cargando, Descargando
        
    def descargar(self, nuevo_destino):
        self.toneladas = 0
        self.origen = self.destino
        self.destino = nuevo_destino
        self.descargas[(origen, destino)] = self.carga 
        self.carga = 0
        return None
    
    def cargar(self, toneladas_cargadas, nuevo_destino):
        self.origen = self.destino
        self.destino = nuevo_destino
        self.carga = toneladas_cargadas
        return None
    
    def asignar_destino(self, Reloj, nuevo_destino, MTV):
        self.destino = nuevo_destino
        actualizar_estado("Viajando", Reloj + MTV[self.origen][self.destino])
    
    def actualizar_estado(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
        
        
        
        

## 1.3. Stock - Botadero

In [6]:
class Stock:
    
    def __init__(self, ID, espacios, ubicacion, tiempo_atencion, tiempo_aculatado):
        self.ID = ID
        self.fila = deque()
        self.espacios_aculatamiento = [0 for i in range(espacios)]
        self.ubicacion = ubicacion
        self.tiempo_atencion = tiempo_atencion
        self.hora_siguiente = 99999999
        self.tiempo_aculatado = tiempo_aculatado 
    
    def avanzar_fila(self):
        camion = self.fila.pop()
        return camion
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
   

## 1.4. Chancador

In [7]:
class Chancador:
    
    def __init__(self, ID, ubicacion, media_falla, media_recuperacion, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ID = ID
        self.ubicacion = ubicacion
        self.espacios_aculatamiento = [0, 0]
        self.fila = deque()
        self.media_falla = media_falla
        self.media_recuperacion = media_recuperacion
        self.tiempo_aculatado = tiempo_aculatado
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
    
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        return(min(horas_eventos.values()))
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
    

# 2. Simulación
## 2.1. Inicialización 

In [17]:
#Se evalúa que palas estarán operativas y en que condición y se generan las instancias 
L_Entidades = []
#Esteril: 303
#Mineral: 293
D_Palas = {}
for pala in InfoPalas:
    u = np.random.uniform(0,1)
    
    if 0 <= u and u < pala[3]:
        pass
    
    elif pala[3] <= u and u < pala[4]:
        D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", "NODO", pala[2], pala[1], "velocidad", \
                                pala[8], pala[5], pala[6], pala[7])
        L_Entidades.append(D_Palas[pala[0]])
       
    else:
        D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", "NODO", pala[2], pala[1], "velocidad", \
                                pala[8], pala[5], pala[6], pala[7]) 
        L_Entidades.append(D_Palas[pala[0]])
#print("Palas: ",D_Palas)

#Se inicializan los camiones
D_Camiones = {}
for i in range(total_camiones):
    D_Camiones[i] = Camion(i)
    L_Entidades.append(Camion(i))
#print("Camiones :", D_Camiones)

#Se inicializan los stocks/botaderos
D_Stocks = {}
for stock in SoB:
    D_Stocks[stock] = Stock(stock, espacios, "NODO", tiempo_atencion_stock, "TIEMPO ACULATADO")
    L_Entidades.append(D_Stocks[stock])
#print("Stocks y Botaderos :", D_Stocks)    

#Se inicializa el Chancador
D_Chancador ={}
for chancador in InfoChancador:
    D_Chancador[chancador[0]] = Chancador(chancador[0], "NODO", chancador[1], chancador[2], chancador[8], \
                                          chancador[5], chancador[6], chancador[7])
    L_Entidades.append(D_Chancador[chancador[0]])
#print("Chancador: ", D_Chancador)

print(L_Entidades)

[PA21, PA13, PA12, PA15, PA9, PA10, PA6, PA8, PA20, PA11, PA14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, S1, S2, B1, B2, CS03]


In [10]:
#Identificamos el siguiente evento que va a ocurrir



D_Camiones[0].__class__.__name__

'Camion'